### Vector stores and retrievers
LangChain's vector store and retriever abstractions. These abstractions are designed to support retrieval of data-- from (vector) databases and other sources-- for integration with LLM workflows. They are important for applications that fetch data to be reasoned over as part of model inference, as in the case of retrieval-augmented generation.

We will cover 
- Documents
- Vector stores
- Retrievers


### Documents
LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content;
- metadata: a dict containing arbitrary metadata.
The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [1]:
from langchain_core.documents import Document

In [2]:
documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [3]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [5]:
from langchain_groq import ChatGroq

In [6]:
model = ChatGroq(model="Llama3-8b-8192", groq_api_key=groq_api_key)

In [7]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026536126BD0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026536269E10>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\004NQ8744\OneDrive\Roadwork\langChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
# VectorStores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)

In [12]:
vectorstore

In [13]:
vectorstore.similarity_search("cat")

[Document(id='407fc438-c1b2-43de-ba38-41891cd350a7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='3ce91957-c185-47c1-bff2-2cf05ea00643', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='1f9055b0-3c10-4a03-9488-f07d8a6c4d35', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='051f9130-d5ba-45aa-a351-31daf7f1717f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [15]:
# Async query search
await vectorstore.asimilarity_search("cat")

[Document(id='407fc438-c1b2-43de-ba38-41891cd350a7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(id='3ce91957-c185-47c1-bff2-2cf05ea00643', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='1f9055b0-3c10-4a03-9488-f07d8a6c4d35', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(id='051f9130-d5ba-45aa-a351-31daf7f1717f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [16]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='407fc438-c1b2-43de-ba38-41891cd350a7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.9351056218147278),
 (Document(id='3ce91957-c185-47c1-bff2-2cf05ea00643', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  1.5740898847579956),
 (Document(id='1f9055b0-3c10-4a03-9488-f07d8a6c4d35', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  1.595690369606018),
 (Document(id='051f9130-d5ba-45aa-a351-31daf7f1717f', metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  1.665792465209961)]

### Retrievers
LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language (LCEL) chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [17]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)

In [20]:
retriever.batch(["cat","dog"])

[[Document(id='407fc438-c1b2-43de-ba38-41891cd350a7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='3ce91957-c185-47c1-bff2-2cf05ea00643', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [21]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1},
)

In [22]:
retriever.batch(["cat","dog"])

[[Document(id='407fc438-c1b2-43de-ba38-41891cd350a7', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(id='3ce91957-c185-47c1-bff2-2cf05ea00643', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]]